In [32]:
import pandas as pd
import json
import tweepy
pd.options.display.max_colwidth=100

In [2]:
import yaml

In [25]:
with open("access_config.yaml") as f:
    access_dict = yaml.safe_load(f)

In [26]:
access_dict

{'api_key': '7YkO1GQdXoMn8Y2uRMjlyehib',
 'api_key_secret': 'jKgIgPapp6kc4jALa6Nhui4q0iTK1TcHk5D641pSn1DuNL2eHF',
 'access_token': '1390221257384333312-ZORIsocGAcEiCszcTCoLcKx8IBIdQx',
 'access_token_secret': 'CyL8XpAhFy59aRRxXC2uwFo8RdyiV7pJcxlEGnbEH0GVb',
 'bearer_token': 'AAAAAAAAAAAAAAAAAAAAAFKljwEAAAAAvYAfpgIn562Ns7HJWR8cqbdKE9c%3DfIlfHrvXHnWOcU4CHNmc9BtueV2Ki5hdvQx7LEAUr7VcYVOr5V'}

In [27]:
# authentication
api_key = access_dict['api_key']
api_key_secret = access_dict['api_key_secret']
access_token = access_dict['access_token']
access_token_secret = access_dict['access_token_secret']
bearer_token = access_dict['bearer_token']

In [7]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [8]:
api = tweepy.API(auth)

In [11]:
public_tweets = api.home_timeline()

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [126]:
v2_client = tweepy.Client(bearer_token=bearer_token, consumer_key=api_key, consumer_secret=api_key_secret, access_token=access_token, access_token_secret=access_token_secret, return_type=dict)

In [165]:
# query = 'from:elonmusk -is:retweet'
query = '#biden lang:en -is:retweet'

tweets = v2_client.search_recent_tweets(
    query=query,
    max_results = 100,
    start_time='2022-11-28T00:00:00.000Z',
    end_time='2022-11-28T23:59:59.000Z',
    tweet_fields = ['author_id','created_at','text','source','lang','geo'],
    user_fields = ['name','username','location','verified'],
    expansions = ['geo.place_id', 'author_id'],
    place_fields = ['country','country_code']
)

In [171]:
pd.DataFrame(data=tweets['data'],
             columns=['author_id', 'created_at', 'lang', 'id', 'source', 'text']
             )

,author_id,created_at,lang,id,source,text
0,794332494489325568,2022-11-28T23:58:08.000Z,en,1597379345340694528,Twitter Web App,@TomFitton @elonmusk @Twitter Not a problem of concern for #Biden &amp; #Dems -- #BorderCrisis ...
1,581782217,2022-11-28T23:57:47.000Z,en,1597379257625346053,CliffordRibnerWeb,Barr Deliberately Sabotaged The 2020 Election Fraud Investigation. Why? #barr #biden #democrats ...
2,1009517269658030082,2022-11-28T23:57:35.000Z,en,1597379208912961536,dlvr.it,Funny Obey #Biden Meme Anti Government #Political Humor #Trump Ultra Maga T #Shirt https://t.co/...
3,1009517269658030082,2022-11-28T23:57:34.000Z,en,1597379202139189249,dlvr.it,Funny Rainbow Let's Go Brandon FJB Anti #Biden Politics Ultra Maga #Trump T #Shirt https://t.co/...
4,1009517269658030082,2022-11-28T23:57:32.000Z,en,1597379195197607936,dlvr.it,#Biden Let's Go Brandon #Political Funny Graphic USA Distressed Cotton T #Shirt https://t.co/VE6...
...,...,...,...,...,...,...
95,15484198,2022-11-28T22:53:11.000Z,en,1597362998989197312,Twitter for iPhone,"PATREON PRIORITY ACCESS\n#MOATS PODCAST\n\n#191 - “Frontline Ukraine, Taiwan elections and Arab ..."
96,1477660185640402944,2022-11-28T22:52:49.000Z,en,1597362907137703937,Twitter Web App,Decisions made by the cognitively impaired old turnip in the #OvalOffice continue to make NO SEN...
97,1445142037762805761,2022-11-28T22:52:19.000Z,en,1597362781703266304,Twitter for iPhone,#Biden is bought and paid for by #CommunistChina and #Ukraine.
98,1009517269658030082,2022-11-28T22:51:32.000Z,en,1597362586873638912,dlvr.it,"Iranian activist has message for #Biden, world leaders https://t.co/FTsJEPsKLS"


In [213]:
more_tweets = tweepy.Paginator(v2_client.search_recent_tweets,
                 query=query,
                 max_results = 100,
                 start_time='2022-11-29T00:00:00.000Z',
                 end_time='2022-11-29T23:59:59.000Z',
                 tweet_fields = ['author_id','created_at','text','source','lang','geo'],
                 user_fields = ['name','username','location','verified'],
                 expansions = ['geo.place_id', 'author_id'],
                 place_fields = ['country','country_code']).flatten(limit=10000)

In [214]:
df = pd.DataFrame(data=list(more_tweets),
                  columns=['author_id', 'created_at', 'lang', 'id', 'source', 'text']
)

In [215]:
df.to_csv('biden_tweets_raw_29_11.csv', index=False)

In [221]:
a = '2022-11-30T00:00:00.000Z'
a.split('T')[0]

'2022-11-30'

In [222]:
a = [['a', 'b'], ['c'], ['a', 'b']]

In [228]:
a.

TypeError: unhashable type: 'list'